# Interfacing with the shell
Python can be used to automate various tasks. This generally requires calling other programs or invoking shell commands.
There are two modules that help making this easier: `argparse` and `invoke`. The former is a standard library module, while the other must be installed (either via pip or another package manager).

We will now go into a couple of examples of how to use them and then propose some exercises.
Useful documentation can be found at
- [argparse](https://docs.python.org/3/library/argparse.html)
- [invoke](http://www.pyinvoke.org/)

In [1]:
# This is a simple argparse example
# The argparse module is much more flexible than this, but this is the bare minimum to get
# useful results.

import argparse as ap
from enum import Enum

class Gender(Enum):
    MALE = "male"
    FEMALE = "female"
    PREFER_NOT_SAY = "prefer-not-say"
    
    def __str__(self):
        return self.value

def main(args):
    parser = ap.ArgumentParser(description="Example command line interface")
    parser.add_argument("name", type=str, help="This is a mandatory positional argument")
    parser.add_argument("surname", type=str, help="This is another positional argument")
    parser.add_argument("-a", "--age", type=int, help="This is a flag argument, requiring an integer value, it has both a short and a long spelling", required=True)
    parser.add_argument("-g", "--gender", type=Gender, choices=list(Gender), default=Gender.PREFER_NOT_SAY,
                        help="This is a fancy optional choice with a default value, the use of enum is not needed, it can also be done with plain strings or anything else really")
    parser.add_argument("--hobby", type=str, nargs="+", help="This is an optional value accepting 1 or more values")
    
    # if you call this as parser.parse_args() it will implicitly use sys.argv as argument source.
    # We can not do that here as we are in a notebook, if it was called from command line it would pick the arguments without having to give a list
    parsed = parser.parse_args(args)
    
    hello = []
    hello.append("Hello {} {}".format(parsed.name, parsed.surname))
    hello.append("you are {} years old".format(parsed.age))
    if parsed.gender != Gender.PREFER_NOT_SAY:
        hello.append("and your gender is {}".format(parsed.gender))
    if parsed.hobby:
        # parsed.hobby will be a list
        hello.append("hobbies:")
        hello.append(",".join(parsed.hobby))
    print(*hello)

In [16]:
# If this was a program called main.py it would have been called as
# ./main.py Mickey Mouse -a 92 -g male
main(["Mickey", "Mouse", "-a", "92", "-g", "male"])

Hello Mickey Mouse you are 92 years old and your gender is male


In [18]:
main(["Roger", "Smith", "-a", "600", "--hobby", "being an alien", "dressing up"])

Hello Roger Smith you are 600 years old hobbies: being an alien,dressing up


In [19]:
# If and invalid set of arguments is given, or the -h option is given, the parsing fails and an usage string is printed out
main(["Roger"])

usage: ipykernel_launcher.py [-h] -a AGE [-g {male,female,prefer-not-say}]
                             [--hobby HOBBY [HOBBY ...]]
                             name surname
ipykernel_launcher.py: error: the following arguments are required: surname, -a/--age


SystemExit: 2

## Exercise 1
Use argparse to create a script that mimics the interface to an email system. It should have the following features:
1. The user must specify an username and a password (Note that for the purposes of this exercise it is fine to use a command line argument for the password, although it is a bad idea to do so because it will be saved in plain text into the shell history)
    He does so via positional arguments #0 and #1.
2. The user selects an operation between: sending, receiving or deleting messages. He does so using a command for the choices "send", "receive", "delete".
    **Note: look up the subparsers in the documentation linked above.**
3. When sending he must specify also the --message option with the message and optionally --subject with the subject
4. When receiving he must not specify any other option
5. When deleting he may specify the number of the message to delete via --msgnum, if not all messages are deleted

You do not need to implement the functionality behind all of this, just use the following skeleton as a sample.

In [86]:
## Skeleton functions

import random
import lorem

messages = []

def send(args):
    print("Sending subject={} {}".format(args.subject, args.message))
    messages.append((args.subject, args.message))
    
def recv(args):
    for _ in range(random.randint(1, 5)):
        messages.append((lorem.sentence(), lorem.paragraph()))
    print("Got messages:")
    for i,(s,m) in enumerate(messages):
        print(i, "[{}]".format(s), m)

def delete(args):
    if args.msgnum and args.msgnum > 0 and args.msgnum < len(messages):
        del messages[args.msgnum]
    else:
        messages.clear()

In [87]:
## A solution

class Ops(Enum):
    SEND = "send"
    RECEIVE = "receive"
    DELETE = "delete"
    
    def __str__(self):
        return self.value


def solution1(args):
    parser = ap.ArgumentParser(description="Solution #1")
    parser.add_argument("username", type=str, help="The user name")
    parser.add_argument("password", type=str, help="The password")
    # Sadly we can't use type=Ops here, there are workarounds but they are all ugly
    sub = parser.add_subparsers(dest="operation", help="Operations", required=True)
    
    sender = sub.add_parser(str(Ops.SEND), help="Send messages")
    sender.add_argument("--subject", type=str, help="Subject")
    sender.add_argument("--message", type=str, required=True, help="The message")

    receiver = sub.add_parser(str(Ops.RECEIVE), help="Receive messages")

    deleter = sub.add_parser(str(Ops.DELETE), help="Delete messages")
    deleter.add_argument("--msgnum", type=int, help="Message number to trash")
    
    parsed = parser.parse_args(args)
    parsed.operation = Ops(parsed.operation)
    
    if parsed.operation == Ops.SEND:
        send(parsed)
    elif parsed.operation == Ops.RECEIVE:
        recv(parsed)
    elif parsed.operation == Ops.DELETE:
        delete(parsed)
    else:
        assert False, "Should not be reached"

In [88]:
solution1(["uname", "MyPa$$w0rd", "send", "--message", "Hello there"])
solution1(["uname", "MyPa$$w0rd", "delete"])
solution1(["uname", "MyPa$$w0rd", "receive"])
solution1(["uname", "MyPa$$w0rd", "delete", "--msgnum", "1"])

Sending subject=None Hello there
Got messages:
0 [Tempora quiquia dolor magnam.] Labore dolor ut consectetur magnam ipsum voluptatem. Neque sed dolorem labore adipisci consectetur non ut. Tempora ut ut dolore magnam voluptatem consectetur. Aliquam porro etincidunt etincidunt dolorem ut amet. Dolorem labore tempora est aliquam non.
1 [Eius quisquam aliquam quaerat labore dolore.] Voluptatem adipisci non non dolor tempora velit. Modi etincidunt ut consectetur ipsum. Consectetur etincidunt ut neque aliquam est porro sed. Velit dolorem quiquia neque ut ut ut. Porro labore consectetur labore magnam sed. Eius dolore ut adipisci. Quisquam numquam dolorem non voluptatem dolore sit modi.
